<a href="https://colab.research.google.com/github/Tessellate-Imaging/Monk_Object_Detection/blob/master/example_notebooks/10_pytorch_efficientdet/Train%20-%20with%20validation%20dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# About the network

1. Paper on EfficientDet: https://arxiv.org/abs/1911.09070

2. Blog 1 on EfficientDet: https://towardsdatascience.com/efficientdet-scalable-and-efficient-object-detection-review-4472ffc34fd9

3. Blog 2 on EfficientDet: https://medium.com/@nainaakash012/efficientdet-scalable-and-efficient-object-detection-ea05ccd28427

# Table of contents


## 1. Installation Instructions


## 2. Use trained model to detect animals in images


## 3. How to train using cameratrap dataset

# Installation

 - Run these commands
     
     - git clone https://github.com/Tessellate-Imaging/Monk_Object_Detection.git
     
     - cd Monk_Object_Detection/10_pytorch_efficientdet/installation
     
 - Select the right requirements file and run
 
     - cat requirements_cuda9.0.txt | xargs -n 1 -L 1 pip install

In [ ]:
! git clone https://github.com/Tessellate-Imaging/Monk_Object_Detection.git

In [ ]:
# For colab use the command below
! cd Monk_Object_Detection/10_pytorch_efficientdet/installation && cat requirements_colab.txt | xargs -n 1 -L 1 pip install
# Restart colab runtime now

# For Local systems and cloud select the right CUDA version
# ! cd Monk_Object_Detection/10_pytorch_efficientdet/installation && cat requirements_cuda9.0.txt | xargs -n 1 -L 1 pip install

# Use already trained model for demo

In [ ]:
import os
import sys
sys.path.append("Monk_Object_Detection/10_pytorch_efficientdet/lib/")

In [ ]:
from infer_detector import Infer

In [ ]:
gtf = Infer();

In [ ]:
# Download the pretrained model

In [ ]:
! wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1WNCn1wy3le8aX44YJSqe8mDqkSXiYqpw' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1WNCn1wy3le8aX44YJSqe8mDqkSXiYqpw" -O obj_cameratrap_2_trained.zip && rm -rf /tmp/cookies.txt

In [ ]:
! unzip -qq obj_cameratrap_2_trained.zip

In [ ]:
f = open("trained_weights/custom/classes.txt", 'r');
classes_list = f.readlines();
f.close();

for i in range(len(classes_list)):
    classes_list[i] = classes_list[i][:len(classes_list[i])-1]

In [ ]:
model_path = "trained_weights/custom/efficientdet-d3_trained.pth";
gtf.load_model(model_path, classes_list, use_gpu=True);

In [ ]:
gtf.predict("trained_weights/test/1.jpg", threshold=0.8);
from IPython.display import Image
Image(filename='output.jpg')

In [ ]:
gtf.predict("trained_weights/test/2.jpg", threshold=0.8);
from IPython.display import Image
Image(filename='output.jpg')

In [ ]:
gtf.predict("trained_weights/test/3.jpg", threshold=0.8);
from IPython.display import Image
Image(filename='output.jpg')

In [ ]:
gtf.predict("trained_weights/test/4.jpg", threshold=0.8);
from IPython.display import Image
Image(filename='output.jpg')

In [ ]:
gtf.predict("trained_weights/test/5.jpg", threshold=0.8);
from IPython.display import Image
Image(filename='output.jpg')

In [ ]:
gtf.predict("trained_weights/test/6.jpg", threshold=0.8);
from IPython.display import Image
Image(filename='output.jpg')

# Train your own detector

## Dataset credits
    - Download link: https://www.dropbox.com/s/e6akva1ynzu7mha/cameratrap.rar?dl=0
    
### Unzip the file innside a new directory data

In [ ]:
# Sorting the data 

In [ ]:
import os
import numpy as np
import cv2
import sys

In [ ]:
os.listdir("data")

In [ ]:
len(os.listdir("data/images"))

In [ ]:
f = open("data/groundtruth.txt", 'r');
lines = f.readlines();
f.close();

In [ ]:
len(lines)

In [ ]:
os.mkdir("data/train_images");

In [ ]:
from tqdm import tqdm

combined = [];

for i in tqdm(range(len(lines))):
    tmp = lines[i].split("\t");
    
    try:
        img_name = tmp[0].split("/")[-1];
        num_anno = int(tmp[1]);

        if(os.path.isfile("data/images/" + img_name)):
            if(num_anno > 0):
                wr = "";
                for j in range(2, 2 + num_anno):
                    label = int(tmp[j].split()[0]);
                    x1 = int(tmp[j].split()[1]);
                    y1 = int(tmp[j].split()[2]);
                    x2 = int(tmp[j].split()[3]);
                    y2 = int(tmp[j].split()[4]);

                    if(label == 1):
                        label = "animal";
                    else:
                        label = "human";
                    wr += str(x1) + " " + str(y1) + " " + str(x2) + " " + str(y2) + " " + label + " ";
                wr = wr[:len(wr)-1];
                os.system("cp data/images/" + img_name + " data/train_images/");
                combined.append([img_name, wr]);
    except:
        print(tmp);
        

In [ ]:
len(combined)

In [ ]:
combined[:5]

In [ ]:
import pandas as pd
df = pd.DataFrame(combined, columns = ['Id', 'Label']) 

df.to_csv("data/train_labels.csv", index=False);

In [ ]:
len(os.listdir("data/train_images"))

In [ ]:
import os
import numpy as np 
import cv2
import dicttoxml
import xml.etree.ElementTree as ET
from xml.dom.minidom import parseString
from tqdm import tqdm
import shutil
import json
import pandas as pd

In [ ]:
root = "data/";
img_dir = "train_images/";
anno_file = "train_labels.csv";

dataset_path = root;
images_folder = root + "/" + img_dir;
annotations_path = root + "/annotations/";

if not os.path.isdir(annotations_path):
    os.mkdir(annotations_path)
    
input_images_folder = images_folder;
input_annotations_path = root + "/" + anno_file;


output_dataset_path = root;
output_image_folder = input_images_folder;
output_annotation_folder = annotations_path;

tmp = img_dir.replace("/", "");
output_annotation_file = output_annotation_folder + "/instances_" + tmp + ".json";
output_classes_file = output_annotation_folder + "/classes.txt";

if not os.path.isdir(output_annotation_folder):
    os.mkdir(output_annotation_folder);
    
df = pd.read_csv(input_annotations_path);
columns = df.columns

delimiter = " ";

list_dict = [];
anno = [];
for i in range(len(df)):
    img_name = df[columns[0]][i];
    labels = df[columns[1]][i];
    tmp = labels.split(delimiter);
    for j in range(len(tmp)//5):
        label = tmp[j*5+4];
        if(label not in anno):
            anno.append(label);
    anno = sorted(anno)
    
for i in tqdm(range(len(anno))):
    tmp = {};
    tmp["supercategory"] = "master";
    tmp["id"] = i;
    tmp["name"] = anno[i];
    list_dict.append(tmp);

anno_f = open(output_classes_file, 'w');
for i in range(len(anno)):
    anno_f.write(anno[i] + "\n");
anno_f.close();

coco_data = {};
coco_data["type"] = "instances";
coco_data["images"] = [];
coco_data["annotations"] = [];
coco_data["categories"] = list_dict;
image_id = 0;
annotation_id = 0;


for i in tqdm(range(len(df))):
    img_name = df[columns[0]][i];
    labels = df[columns[1]][i];
    tmp = labels.split(delimiter);
    image_in_path = input_images_folder + "/" + img_name;
    img = cv2.imread(image_in_path, 1);
    h, w, c = img.shape;

    images_tmp = {};
    images_tmp["file_name"] = img_name;
    images_tmp["height"] = h;
    images_tmp["width"] = w;
    images_tmp["id"] = image_id;
    coco_data["images"].append(images_tmp);
    

    for j in range(len(tmp)//5):
        x1 = int(tmp[j*5+0]);
        y1 = int(tmp[j*5+1]);
        x2 = int(tmp[j*5+2]);
        y2 = int(tmp[j*5+3]);
        label = tmp[j*5+4];
        annotations_tmp = {};
        annotations_tmp["id"] = annotation_id;
        annotation_id += 1;
        annotations_tmp["image_id"] = image_id;
        annotations_tmp["segmentation"] = [];
        annotations_tmp["ignore"] = 0;
        annotations_tmp["area"] = (x2-x1)*(y2-y1);
        annotations_tmp["iscrowd"] = 0;
        annotations_tmp["bbox"] = [x1, y1, x2-x1, y2-y1];
        annotations_tmp["category_id"] = anno.index(label);

        coco_data["annotations"].append(annotations_tmp)
    image_id += 1;

outfile =  open(output_annotation_file, 'w');
json_str = json.dumps(coco_data, indent=4);
outfile.write(json_str);
outfile.close();

## Training 

In [ ]:
import os
import sys
sys.path.append("Monk_Object_Detection/10_pytorch_efficientdet/lib/")

In [ ]:
from train_detector import Detector

In [ ]:
gtf = Detector();

In [ ]:
f = open("data/annotations/classes.txt", 'r');
class_list = f.readlines();
f.close();

In [ ]:
root_dir = "data/";
coco_dir = "";
img_dir = "";
set_dir = "train_images";

In [ ]:
gtf.set_train_dataset(root_dir, coco_dir, img_dir, set_dir, classes_list=class_list, batch_size=2, num_workers=3)

In [ ]:
gtf.set_model(model_name="efficientdet-d3.pth", num_gpus=1, freeze_head=False)

In [ ]:
gtf.set_hyperparams(optimizer="adamw", lr=0.0001, es_min_delta=0.0, es_patience=0)

In [ ]:
gtf.train(num_epochs=10, val_interval=1, save_interval=1)

## Inference

In [ ]:
import os
import sys
sys.path.append("Monk_Object_Detection/10_pytorch_efficientdet/lib/")

In [ ]:
from infer_detector import Infer

In [ ]:
gtf = Infer();

In [ ]:
f = open("data/annotations/classes.txt", 'r');
class_list = f.readlines();
f.close();

In [ ]:
class_list

In [ ]:
model_path = "trained_weights/custom/efficientdet-d3_trained.pth";
gtf.load_model(model_path, class_list, use_gpu=True);

In [ ]:
img_list = os.listdir("data/train_images/")

In [ ]:
len(img_list)

In [ ]:
img_path = "data/train_images/" + img_list[7]
gtf.predict(img_path, threshold=0.5)

from IPython.display import Image
Image(filename='output.jpg')

In [ ]:
img_path = "data/train_images/" + img_list[10]
gtf.predict(img_path, threshold=0.5)

from IPython.display import Image
Image(filename='output.jpg')

In [ ]:
img_path = "data/train_images/" + img_list[150]
gtf.predict(img_path, threshold=0.5)

from IPython.display import Image
Image(filename='output.jpg')

In [ ]:
img_path = "data/train_images/" + img_list[250]
gtf.predict(img_path, threshold=0.5)

from IPython.display import Image
Image(filename='output.jpg')